# PVAPy Exploration

My goals are to do simple channel access and pvaccess gets and monitors

In [1]:
import pvaccess as pva
import datetime
import time
from time import sleep
import numpy as np

## Channel Access Gets

Drawing on examples folder from GitHub for PVAPy, especially the [channelMonitorExample.py](https://github.com/epics-base/pvaPy/blob/master/examples/channelMonitorExample.py)

### Create a Channel object

In [2]:
c = pva.Channel('PULSEGEN:info', pva.CA)
type(c)

pvaccess.pvaccess.Channel

### Get and process just the value for this channel

#### Retrieve a PVObject object by calling "get" on the channelpvobj.getAsString()

In [3]:
pvobj = c.get()
type(pvobj)

pvaccess.pvaccess.PvObject

#### View the PVObject as a string

In [4]:
pvobj.getAsString()

'DOLPHINDAQ,TeensyPulse,00,20240807'

#### View the PVObject in its dict form

In [5]:
pvobj.get()

{'value': 'DOLPHINDAQ,TeensyPulse,00,20240807'}

#### Generic conversion into the right format

In [6]:
pvobj.getPyObject()

'DOLPHINDAQ,TeensyPulse,00,20240807'

### Get and process both the field and timestamp

In [7]:
pvobj = c.get('field(value,timeStamp)')
assert(isinstance(pvobj, pva.pvaccess.PvObject))

In [8]:
pvobj.get()

{'value': 'DOLPHINDAQ,TeensyPulse,00,20240807',
 'timeStamp': {'secondsPastEpoch': 1724724463,
  'nanoseconds': 798206584,
  'userTag': 0}}

In [9]:
pvobj.getPyObject('timeStamp')

{'secondsPastEpoch': 1724724463, 'nanoseconds': 798206584, 'userTag': 0}

In [10]:
type(pvobj.getPyObject('timeStamp'))

dict

In [11]:
pvobj["timeStamp"]["secondsPastEpoch"]

1724724463

In [12]:
# pvobj_ts = pva.PvTimeStamp(pvobj["timeStamp"]["secondsPastEpoch"], pvobj["timeStamp"]["nanoseconds"])

In [13]:
pvobj_ts = datetime.datetime.fromtimestamp(pvobj["timeStamp"]["secondsPastEpoch"] + pvobj["timeStamp"]["nanoseconds"]*1e-9)

In [14]:
pvobj.getStructureDict()

{'value': pvaccess.pvaccess.ScalarType.STRING,
 'timeStamp': {'secondsPastEpoch': pvaccess.pvaccess.ScalarType.LONG,
  'nanoseconds': pvaccess.pvaccess.ScalarType.INT,
  'userTag': pvaccess.pvaccess.ScalarType.INT}}

In [15]:
def myget(channel):
    """ Synchronous get on a channel, with conversion to regular python objects """
    pvobj = channel.get('field(value,timeStamp)')
    value = pvobj["value"]
    timestamp = datetime.datetime.fromtimestamp(pvobj["timeStamp"]["secondsPastEpoch"] + pvobj["timeStamp"]["nanoseconds"]*1e-9)
    return value, timestamp

In [16]:
myproc = pva.Channel('PULSEGEN:info.PROC', pva.CA)
myproc.get().get()

{'value': '\x00'}

In [17]:
myproc.put

<bound method put of <pvaccess.pvaccess.Channel object at 0x000001F1A2CBF150>>

### Trying to get another field besides value, timeStamp, alarm

In [18]:
pvobj = c.get()
pvobj.get()

{'value': 'DOLPHINDAQ,TeensyPulse,00,20240807'}

In [19]:
pvobj = c.get('value,alarm,timeStamp,display,control,valueAlarm')
pvobj.get()

{'value': 'DOLPHINDAQ,TeensyPulse,00,20240807',
 'alarm': {'severity': 0, 'status': 0, 'message': ''},
 'timeStamp': {'secondsPastEpoch': 1724724473,
  'nanoseconds': 798219342,
  'userTag': 0}}

## Trying to only get fresh data

#### Delays show the value is changing

In [20]:
trigcnt = pva.Channel('PULSEGEN:trigger:count', pva.CA)
for i in range(10):
    value, timestamp = myget(trigcnt)
    print(timestamp, ": ", value)
    sleep(1)

2024-08-26 19:07:54.823354 :  3167.0
2024-08-26 19:07:55.823369 :  3177.0
2024-08-26 19:07:56.823333 :  3187.0
2024-08-26 19:07:57.823359 :  3197.0
2024-08-26 19:07:58.823338 :  3207.0
2024-08-26 19:07:59.823326 :  3217.0
2024-08-26 19:08:00.823365 :  3227.0
2024-08-26 19:08:01.823376 :  3237.0
2024-08-26 19:08:02.823361 :  3247.0
2024-08-26 19:08:03.823346 :  3257.0


#### Without delays, no changes

In [21]:
trigcnt = pva.Channel('PULSEGEN:trigger:count', pva.CA)
for i in range(10):
    value, timestamp = myget(trigcnt)
    print(timestamp, ": ", value)

2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0


#### With minimal delays, streaks and then changes
Record is not being forced to process. Updating at the scan rate of the record, which is set to "1 second":

```
record(int64in,"$(P):trigger:count"){
    field(DESC,"Get global trigger count")
    field(SCAN,"1 second")
	field(DTYP,"stream")
    field(INP,"@teensypulse.proto TriggerCountQ $(BUS)")
    }
```

In [22]:
trigcnt = pva.Channel('PULSEGEN:trigger:count', pva.CA)
for i in range(10):
    value, timestamp = myget(trigcnt)
    print(timestamp, ": ", value)
    sleep(0.5)

2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:04.823331 :  3267.0
2024-08-26 19:08:05.821527 :  3277.0
2024-08-26 19:08:05.821527 :  3277.0
2024-08-26 19:08:06.823338 :  3287.0
2024-08-26 19:08:06.823338 :  3287.0
2024-08-26 19:08:07.823392 :  3297.0
2024-08-26 19:08:07.823392 :  3297.0
2024-08-26 19:08:08.823540 :  3307.0
2024-08-26 19:08:08.823540 :  3307.0


#### Force a record to process faster than its scan rate

Reference: [EPICS Tech-talk 2012 question about forcing processing](https://epics.anl.gov/tech-talk/2012/msg01561.php)

In [23]:
trigcnt = pva.Channel('PULSEGEN:trigger:count', pva.CA)
trigcnt_proc = pva.Channel('PULSEGEN:trigger:count.PROC', pva.CA)
for i in range(10):
    trigcnt_proc.put(1) # Force record to process
    value, timestamp = myget(trigcnt)
    print(timestamp, ": ", value)
    sleep(0.05)

2024-08-26 19:08:10.197187 :  3321.0
2024-08-26 19:08:10.197187 :  3321.0
2024-08-26 19:08:10.278386 :  3321.0
2024-08-26 19:08:10.334633 :  3322.0
2024-08-26 19:08:10.388928 :  3322.0
2024-08-26 19:08:10.504403 :  3324.0
2024-08-26 19:08:10.561920 :  3324.0
2024-08-26 19:08:10.618255 :  3325.0
2024-08-26 19:08:10.677672 :  3325.0
2024-08-26 19:08:10.734636 :  3326.0


## How fast can I set a Control Setting? (Answer: 20-30 milliseconds)

In [24]:
powers = pva.Channel('LASER:powers:set', pva.CA)
powers_RBV = pva.Channel('LASER:powers', pva.CA)
powers_RBV_proc = pva.Channel('LASER:powers.PROC', pva.CA)

In [25]:
powers_RBV.get().getScalarArray()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [26]:
ascending = np.uint8(np.linspace(0, 255, 100))
descending = np.uint8(np.linspace(255, 0, 100))
spiky = np.uint8(np.tile([0,255], 50))

In [27]:
powers.putScalarArray(list(spiky))

In [28]:
powers_RBV.get().getScalarArray()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

#### Timer decorator
Code copied directly from https://dev.to/kcdchennai/python-decorator-to-measure-execution-time-54hk

In [29]:
from functools import wraps
import time

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        #print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        print(f'Function {func.__name__} took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [30]:
@timeit
def reach_setpoint(powers_setpoint_array, powers_CA=powers, powers_RBV_CA=powers_RBV):
    """ Gets us to a certain setpoint for a value """
    powers_CA.putScalarArray(list(powers_setpoint_array))
    while not np.array_equal(powers_RBV_CA.get().getScalarArray(), powers_setpoint_array):
        sleep(0.01)
    return

In [31]:
for setpoint in [ascending, descending, spiky]:
    reach_setpoint(setpoint)

Function reach_setpoint took 0.0195 seconds
Function reach_setpoint took 0.0954 seconds
Function reach_setpoint took 0.0983 seconds


In [32]:
@timeit
def reach_setpoint_faster(powers_setpoint_array, powers_CA=powers, powers_RBV_proc_CA=powers_RBV_proc, powers_RBV_CA=powers_RBV):
    """ Gets us to a certain setpoint for a value """
    powers_CA.putScalarArray(list(powers_setpoint_array))
    while not np.array_equal(powers_RBV_CA.get().getScalarArray(), powers_setpoint_array):
        powers_RBV_proc_CA.put(1)
        sleep(0.01)
    return

In [33]:
for setpoint in [ascending, descending, spiky]:
    reach_setpoint_faster(setpoint)

Function reach_setpoint_faster took 0.0345 seconds
Function reach_setpoint_faster took 0.0200 seconds
Function reach_setpoint_faster took 0.0299 seconds


In [34]:
for i in range(10):
    for setpoint in [ascending, descending, spiky]:
        reach_setpoint_faster(setpoint)

Function reach_setpoint_faster took 0.0196 seconds
Function reach_setpoint_faster took 0.0205 seconds
Function reach_setpoint_faster took 0.0206 seconds
Function reach_setpoint_faster took 0.0258 seconds
Function reach_setpoint_faster took 0.0204 seconds
Function reach_setpoint_faster took 0.0246 seconds
Function reach_setpoint_faster took 0.0223 seconds
Function reach_setpoint_faster took 0.0214 seconds
Function reach_setpoint_faster took 0.0294 seconds
Function reach_setpoint_faster took 0.0262 seconds
Function reach_setpoint_faster took 0.0222 seconds
Function reach_setpoint_faster took 0.0211 seconds
Function reach_setpoint_faster took 0.0305 seconds
Function reach_setpoint_faster took 0.0196 seconds
Function reach_setpoint_faster took 0.0208 seconds
Function reach_setpoint_faster took 0.0220 seconds
Function reach_setpoint_faster took 0.0333 seconds
Function reach_setpoint_faster took 0.0207 seconds
Function reach_setpoint_faster took 0.0233 seconds
Function reach_setpoint_faster 

The above seems like a success to me! I love how once in a while the set time took longer, which makes putting a hard-coded delay in here a bad plan. This approach works well.

## PVAccess gets

In [35]:
c = pva.Channel('ELECTRON-DAQ:info', pva.PVA)
c.get().get()

{'value': 'Diode Teensy-Serial PVA Server, 20240724',
 'alarm': {'severity': 0, 'status': 0, 'message': ''},
 'timeStamp': {'secondsPastEpoch': 0, 'nanoseconds': 0, 'userTag': 0}}

In [36]:
c = pva.Channel('ELECTRON-DAQ:trace', pva.PVA)
#pvobj = c.get('field(value,timeStamp,uniqueId,dataTimeStamp)')
#pvobj["dataTimeStamp"]

@timeit
def await_fresh_data(channel):
    """Await a new set of data with timestamp after the current time (relies on precision clock synchronization)"""    
    minimum_timestamp = datetime.datetime.now() # timestamp that must be surpassed in the data before continuing

    while True:
        pvobj = c.get('field(timeStamp)')
        pvtimestamp = datetime.datetime.fromtimestamp(pvobj["timeStamp"]["secondsPastEpoch"] + pvobj["timeStamp"]["nanoseconds"]*1e-9)
        # print(pvtimestamp) # DEBUG
        if pvtimestamp > minimum_timestamp:
            break
        sleep(0.005)

In [37]:
await_fresh_data(c)

Function await_fresh_data took 0.5428 seconds


In [38]:
c.get('field(timeStamp)')

In [39]:
print(pvobj)

structure 
    string value DOLPHINDAQ,TeensyPulse,00,20240807
    alarm_t alarm
        int severity 0
        int status 0
        string message 
    time_t timeStamp
        long secondsPastEpoch 1724724473
        int nanoseconds 798219342
        int userTag 0



### Bring it all together

In [40]:
powers = pva.Channel('LASER:powers:set', pva.CA)
powers_RBV = pva.Channel('LASER:powers', pva.CA)
powers_RBV_proc = pva.Channel('LASER:powers.PROC', pva.CA)
electron_trace = pva.Channel('ELECTRON-DAQ:trace', pva.PVA)
for setpoint in [ascending, descending, spiky]:
    reach_setpoint_faster(setpoint)
    await_fresh_data(electron_trace)
    print(electron_trace.get().get()['value'][0]['ushortValue'])
    print(np.diff(np.float64(electron_trace.get().get()['value'][0]['ushortValue'])))
    #print(electron_trace.get())

Function reach_setpoint_faster took 0.0204 seconds
Function await_fresh_data took 0.5329 seconds
[  0   0   0   0   0   0   0   0   0   0   1   0   3   0   0   0   0   0
   0   0   0   0   0   0   0   0   4  11  23  38  51  57  64  77  90  92
 106 122 128 140 163 168 181 211 222 236 244 250 259 258 260 262 262 264
 264 265 265 267 266 266 267 267 267 268 269 268 269 269 271 270 271 270
 270 270 270 274 270 270 272 271 272 270 270 270 271 271 270 272 272 271
 272 272 273 270 272 273 272 273 273 273]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -1.  3. -3.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  4.  7. 12. 15. 13.  6.  7. 13. 13.  2. 14.
 16.  6. 12. 23.  5. 13. 30. 11. 14.  8.  6.  9. -1.  2.  2.  0.  2.  0.
  1.  0.  2. -1.  0.  1.  0.  0.  1.  1. -1.  1.  0.  2. -1.  1. -1.  0.
  0.  0.  4. -4.  0.  2. -1.  1. -2.  0.  0.  1.  0. -1.  2.  0. -1.  1.
  0.  1. -3.  2.  1. -1.  1.  0.  0.]
Function reach_setpoint_faster took 0.0234 seconds
Function await_fresh_data took 0.5610 seco

In [41]:
electron_trace.get().get()['value'][0]['ushortValue']

array([  0,   0,  21, 205, 124, 210, 141, 233, 124, 213, 138, 238, 156,
       223, 133, 240, 150, 225, 130, 244, 147, 229, 124, 208, 137, 231,
       121, 213, 133, 234, 150, 220, 128, 240, 147, 223, 124, 203, 136,
       225, 120, 206, 133, 230, 150, 214, 128, 232, 146, 218, 125, 238,
       142, 222, 120, 203, 133, 224, 117, 206, 129, 228, 146, 214, 123,
       232, 142, 217, 120, 198, 132, 219, 117, 202, 130, 225, 146, 208,
       124, 227, 143, 212, 121, 231, 139, 216, 117, 197, 129, 219, 114,
       202, 124, 222, 142, 209, 120, 226, 139, 212], dtype=uint16)

In [42]:
reach_setpoint_faster(ascending)

Function reach_setpoint_faster took 0.0197 seconds


In [43]:
data = np.array([1.999,2.0])
